In [3]:
import yfinance as yf # type: ignore
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# --- 1. Pengaturan Tanggal dan Ticker ---
# Mengatur periode 5 tahun terakhir dari hari ini
end_date = pd.Timestamp.now()
start_date = end_date - pd.DateOffset(years=5)

# Ticker untuk S&P 500, IEF, dan 10-Year Treasury Yield
tickers = {
    "S&P 500": "^GSPC",
    "IEF": "IEF",
    "Interest Rate (10Y)": "^TNX"
}

# --- 2. Mengunduh Data Historis ---
data = pd.DataFrame()
for name, ticker in tickers.items():
    # Mengambil data harga penutupan ('Adj Close')
    # Untuk yield Treasury, kita gunakan 'Close' karena sudah dalam bentuk persentase
    series = yf.download(ticker, start=start_date, end=end_date)['Adj Close' if ticker != '^TNX' else 'Close']
    data[name] = series

# Menghilangkan baris dengan data yang tidak lengkap (NaN)
data.dropna(inplace=True)

print("Data Historis (5 baris pertama):")
print(data.head())
print("-" * 35)


# --- 3. Menghitung Perubahan Persentase Harian ---
daily_returns = data.pct_change().dropna()

# Khusus untuk Interest Rate, kita gunakan perubahan absolut (karena sudah dalam persen)
daily_returns['Interest Rate (10Y)'] = data['Interest Rate (10Y)'].diff()
daily_returns.dropna(inplace=True)

print("\nPerubahan Harian (5 baris pertama):")
print(daily_returns.head())
print("-" * 35)


# --- 4. Menghitung Korelasi ---
correlation_matrix = daily_returns.corr()

print("\nMatriks Korelasi (5 Tahun Terakhir):")
print(correlation_matrix)
print("-" * 35)

# Mengambil nilai korelasi spesifik
sp500_interest_corr = correlation_matrix.loc["S&P 500", "Interest Rate (10Y)"]
ief_interest_corr = correlation_matrix.loc["IEF", "Interest Rate (10Y)"]

print(f"Korelasi S&P 500 vs Interest Rate: {sp500_interest_corr:.4f}")
print(f"Korelasi IEF vs Interest Rate: {ief_interest_corr:.4f}")


# --- 5. Visualisasi ---
# Normalisasi data untuk membandingkan pergerakan dalam satu grafik
normalized_data = (data / data.iloc[0]) * 100

plt.style.use('seaborn-v0_8-whitegrid')
fig, ax1 = plt.subplots(figsize=(14, 7))

# Plot S&P 500 dan IEF pada sumbu y kiri
ax1.plot(normalized_data.index, normalized_data['S&P 500'], label='S&P 500 (dinormalisasi)', color='royalblue')
ax1.plot(normalized_data.index, normalized_data['IEF'], label='IEF (dinormalisasi)', color='darkorange')
ax1.set_xlabel('Tanggal')
ax1.set_ylabel('Kinerja yang Dinormalisasi (Basis 100)')
ax1.tick_params(axis='y')
ax1.legend(loc='upper left')

# Membuat sumbu y kedua untuk Interest Rate
ax2 = ax1.twinx()
ax2.plot(data.index, data['Interest Rate (10Y)'], label='Interest Rate (10Y Yield)', color='green', linestyle='--')
ax2.set_ylabel('Imbal Hasil Treasury 10-Tahun (%)')
ax2.tick_params(axis='y')
ax2.legend(loc='upper right')

plt.title('Perbandingan Kinerja S&P 500, IEF, dan Suku Bunga (5 Tahun Terakhir)', fontsize=16)
fig.tight_layout()
plt.show()

# Visualisasi Matriks Korelasi
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=.5)
plt.title('Heatmap Korelasi Perubahan Harian')
plt.show()

ModuleNotFoundError: No module named 'yfinance'